In [1]:
%load_ext autoreload
%autoreload 2
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import numpy as np
from dark_emulator_public import dark_emulator
import os, sys, time, json
import matplotlib.pyplot as plt
from collections import OrderedDict as od
from scipy.interpolate import InterpolatedUnivariateSpline as ius
from scipy.interpolate import interp2d, interp1d
from scipy.integrate import simps
from tqdm import tqdm
import hsc3x2pt

using dark_emulator at  /lustre/work/sunao.sugiyama/package/dark_emulator_public/dark_emulator/__init__.py


## Preparation for Fisher
Computation of $C(l)$ takes ~20 sec for each cosmology.
So this notebook pre-compute $C(l)$ for cosmologies needed for Fisher analysis.

Model parameters of 3x2pt analysis with single(double) source redshift bin are
- comsology: 5
- galaxy bias: 1x3
- magnification bias: 1x3
- photo-z: 1x1(2)
- multiplicative bais: 1x1(2)

13(15) parameters in total.

$C(l)$s depend on cosmological parameters, galaxy bias and magnification bias, for which we need to compute $C(l)$.

**Note** : Fisher analysis is doable even if the computational time for one model reaches ~20 sec. However, in the real analysis, ~20 sec is too much to run sampling code. 
I just want to know how people in cosmic shear, like Hikage-san and Hamana-san, or people in 3x2pt, like DES or KiDS, implement code of $C(l)$.?

In [3]:
power_b1 = hsc3x2pt.power_b1_class()

initialize cosmo_class
Initialize pklin emulator
initialize propagator emulator
Initialize sigma_d emulator
initialize cross-correlation emulator
initialize auto-correlation emulator
Initialize sigmaM emulator
initialize xinl emulator


In [4]:
Omega_s_HSC = 140

### List of models to compute $C(l)$ to compute **single** source bin anlaysis
0. fiducial = Planck cosmology + fiducial galaxy bias, magnification bais + 0 photo-z and multiplicative bias
1. $\omega_b h^2$ +
2. $\omega_c h^2$ +
3. $\Omega_{\rm de}$ +
4. $n_{\rm s}$ +
5. $\ln 10^{10}A_{\rm s}$ +
6. $b_{\rm 1,1}$ + 
7. $b_{\rm 1,2}$ + 
8. $b_{\rm 1,3}$ + 
9. $\alpha_{\rm mag,1}$ + 
10. $\alpha_{\rm mag,2}$ + 
11. $\alpha_{\rm mag,3}$ + 
12. $\Delta z_{\rm ph}$ +
13. $\Delta m$ + = easy. Just multiply $1+\Delta m$ to fiducial.

In [5]:
dirname = 'precomputed_Cl/single_source_ClY1IA'

def compute(params, dir_to_save, compute_lens_cross=False):
    t0 = time.time()
    ombh2, omch2, Ode, sigma8, ns, b11, b12, b13, a1, a2, a3, dzph, dm, A_IA, eta_IA = params
    g_l1 = hsc3x2pt.galaxy_sample_lens_class(['lowz'  , 0.15, 0.30, b11, '40.36e-4 arcmin^-2', a1])
    g_l2 = hsc3x2pt.galaxy_sample_lens_class(['cmass1', 0.47, 0.55, b12, '20.06e-4 arcmin^-2', a2])
    g_l3 = hsc3x2pt.galaxy_sample_lens_class(['cmass2', 0.55, 0.70, b13, '30.12e-4 arcmin^-2', a3])
    g_s12= hsc3x2pt.galaxy_sample_source_IA_class(['s12', 'sourcePzs_Y1/MLZs34.txt', dzph, dm, A_IA, eta_IA, 0.2, '6.6 arcmin^-2'])
    pk2cl = hsc3x2pt.pk2cl_class(power_b1)
    pk2cl.set_galaxy_sample(g_l1)
    pk2cl.set_galaxy_sample(g_l2)
    pk2cl.set_galaxy_sample(g_l3)
    pk2cl.set_galaxy_sample(g_s12)
    # set cosmo
    cosmo_dict = dict(zip(['omega_b', 'omega_c', 'Omega_de', 'ln10p10As', 'n_s', 'w_de'], [ombh2, omch2, Ode, 3.094, ns, -1]))
    pk2cl.set_cosmology_from_dict(cosmo_dict)
    sigma8_temp = pk2cl.pk_class.get_sigma8()
    ln10p10As = 3.094 + 2*np.log(sigma8/sigma8_temp)
    print(f'ln10p10As={ln10p10As}')
    cosmo_dict = dict(zip(['omega_b', 'omega_c', 'Omega_de', 'ln10p10As', 'n_s', 'w_de'], [ombh2, omch2, Ode, ln10p10As, ns, -1]))
    pk2cl.set_cosmology_from_dict(cosmo_dict)
    
    pk2cl.init_pk()
    pk2cl.set_Omega_s({'w':8300, 'gamma_t':Omega_s_HSC, 'xi':Omega_s_HSC}) # HSCY1

    l = np.logspace(-2, 5, 1000)
    with hsc3x2pt.Time():
        pk2cl.compute_all_Cl(l, compute_lens_cross=compute_lens_cross)
    pk2cl.dump_Cl_cache(f'{dirname}/{dir_to_save}', overwrite=True, silent=True)
    t1 = time.time()
    print(f'{t1-t0} sec')

In [6]:
dp = 0.01
file_param = od()

def save_dparam(dparam, dir_to_save):
    fname = os.path.join(dirname, 'file_param.json')
    if os.path.exists(fname):
        file_param = json.load(open(fname, 'r'), object_pairs_hook=od) 
    else:
        file_param = od()
    file_param[dir_to_save] = dparam
    json.dump(file_param, open(fname, 'w'), indent=2)

t_start = time.time()

def get_param(names, param_fiducial, absolute_dp, idx):
    param = copy.deepcopy(param_fiducial)
    if absolute_dp[idx]:
        param[idx] = dp
        dparam = dp
    else:
        param[idx] = param[idx]*(1+dp)
        dparam = param[idx]*dp
    name = names[idx]
    return name, param, dparam

In [7]:
param_fiducial = [0.02225, 0.1198, 0.6844, 0.831, 0.9645, 1.78 , 2.10 , 2.28 , 2.259, 3.563, 3.729, 0.0 , 0.0 , 1.0  , 3.0]
absolute_dp    = [False  , False , False , False, False , False, False, False, False, False, False, True, True, False, False]
names = ['omega_b', 'omega_c', 'Omega_de', 'sigma8', 'ns', 'b1lowz', 'b1cmass1', 'b1cmass2', 'alphamaglowz', 'alphamagcmass1', 'alphamagcmass2', 'dzph', 'dm', 'A_IA', 'eta_IA']
compute(param_fiducial, 'fiducial', compute_lens_cross=True)

ln10p10As=3.097214912859019
:26.444217681884766 sec
26.85903263092041 sec


In [ ]:
for idx in range(len(names)):
    name, param, dparam = get_param(names, param_fiducial, absolute_dp, idx)
    print(f'>>> Computing {name}')
    compute(param, name)
    save_dparam(dparam, name)

>>> Computing omega_b
ln10p10As=3.1004231384103087
:16.984068155288696 sec
17.368055820465088 sec
>>> Computing omega_c
ln10p10As=3.082647998157684
:17.157678842544556 sec
17.54237675666809 sec
>>> Computing Omega_de
ln10p10As=3.092198150550478
:16.967852115631104 sec
17.356382131576538 sec
>>> Computing sigma8
ln10p10As=3.117115574565356
:17.093859910964966 sec
17.472084522247314 sec
>>> Computing ns
ln10p10As=3.0899386610164075
:17.010557413101196 sec
17.39804983139038 sec
>>> Computing b1lowz
ln10p10As=3.097214912859019
:17.09132432937622 sec
17.470704793930054 sec
>>> Computing b1cmass1
ln10p10As=3.097214912859019
:17.08319664001465 sec
17.47083616256714 sec
>>> Computing b1cmass2
ln10p10As=3.097214912859019
:16.99283528327942 sec
17.377151489257812 sec
>>> Computing alphamaglowz
ln10p10As=3.097214912859019
:17.03222680091858 sec
17.41188359260559 sec
>>> Computing alphamagcmass1
ln10p10As=3.097214912859019
:17.062827825546265 sec
17.44679617881775 sec
>>> Computing alphamagcmass2


In [25]:
np.savetxt(os.path.join(dirname, 'param_fiducial.txt'), np.array(param_fiducial))

## Prepare for Fisher analysis

In [26]:
fisher2x2pt = hsc3x2pt.getFisher(dirname, power_b1, probes=['w', 'gamma_t'], 
                                 label='2x2pt IA (Y1, 1bin)')
fisher1x2pt = hsc3x2pt.getFisher(dirname, power_b1, probes=['xi+','xi-'], 
                                 label='Cosmic Shear IA (Y1, 1bin)')
fisher3x2pt = hsc3x2pt.getFisher(dirname, power_b1,
                                 label='3x2pt IA (Y1, 1bin)')

cov.shape = (66, 66), dim(data)=(66,)
cov.shape = (17, 17), dim(data)=(17,)
skip b1lowz because this is lens galaxy param, while probes does not include any lens related probe.
skip b1cmass1 because this is lens galaxy param, while probes does not include any lens related probe.
skip b1cmass2 because this is lens galaxy param, while probes does not include any lens related probe.
skip alphamaglowz because this is lens galaxy param, while probes does not include any lens related probe.
skip alphamagcmass1 because this is lens galaxy param, while probes does not include any lens related probe.
skip alphamagcmass2 because this is lens galaxy param, while probes does not include any lens related probe.
cov.shape = (83, 83), dim(data)=(83,)


In [27]:
import joblib
dirname_fisher = os.path.join('FisherMats/', os.path.basename(dirname))
if not os.path.exists(dirname_fisher):
    os.mkdir(dirname_fisher)
joblib.dump(fisher1x2pt, os.path.join(dirname_fisher, 'fisher1x2pt.cmp'), compress=True)
joblib.dump(fisher2x2pt, os.path.join(dirname_fisher, 'fisher2x2pt.cmp'), compress=True)
joblib.dump(fisher3x2pt, os.path.join(dirname_fisher, 'fisher3x2pt.cmp'), compress=True)

['FisherMats/single_source_ClY1IA/fisher3x2pt.cmp']

In [28]:
print(time.time()-t_start, 'sec')

378.304856300354 sec
